In [1]:
%matplotlib inline
import numpy as np
from settings import Settings
from utils import *
from data_processor import data_processor
from augment_patient import augment_patient

In [4]:
import numpy as np
from settings import Settings
from utils import *
from data_processor import data_processor
from augment_patient import augment_patient

lungPatients, labels = load_data()

size = Settings.size
no_slices = Settings.no_slices

imageData = []
augData = []

for num, patient in enumerate(lungPatients):
    if num % 100 == 0:
        print('Saved -', num)
    print('processing patient # {} - {} out of {}'.format(num, patient, len(lungPatients)))

    try:
        if get_label(patient, labels) == 1:
            print('cancer - {}'.format(num))
            augmented_data = []
            for i in range(Settings.aug_counter):
                augmented_data.append(augment_patient(patient=patient, labels_df=labels, iteration=i, size=size, noslices=no_slices, visualize=False))
            for each in augmented_data:
                augData.append([each['img_data'], each['label'], each['patient']])
    except KeyError as e:
        print('Data is unlabeled')
    except Exception as er:
        print(er)
        
np.save('augmentedDataNew-{}-{}-{}.npy'.format(size, size, no_slices), augData)

Saved - 0
processing patient # 0 - 0015ceb851d7251b8f399e39779d1e7d out of 1595
cancer - 0
processing patient # 1 - 0030a160d58723ff36d73f41b170ec21 out of 1595
processing patient # 2 - 003f41c78e6acfa92430a057ac0b306e out of 1595
processing patient # 3 - 006b96310a37b36cccb2ab48d10b49a3 out of 1595
cancer - 3
processing patient # 4 - 008464bb8521d09a42985dd8add3d0d2 out of 1595
cancer - 4
processing patient # 5 - 0092c13f9e00a3717fdc940641f00015 out of 1595
processing patient # 6 - 00986bebc45e12038ef0ce3e9962b51a out of 1595
processing patient # 7 - 00cba091fa4ad62cc3200a657aeb957e out of 1595
processing patient # 8 - 00edff4f51a893d80dae2d42a7f45ad1 out of 1595
cancer - 8
processing patient # 9 - 0121c2845f2b7df060945b072b2515d7 out of 1595
processing patient # 10 - 013395589c01aa01f8df81d80fb0e2b8 out of 1595
processing patient # 11 - 01de8323fa065a8963533c4a86f2f6c1 out of 1595
processing patient # 12 - 01e349d34c06410e1da273add27be25c out of 1595
processing patient # 13 - 01f1140

processing patient # 111 - 14248ec22193bed1e087755df5a28509 out of 1595
processing patient # 112 - 1427be78bcf4aba96c5054b697be9b5b out of 1595
cancer - 112
processing patient # 113 - 149cc798827099f8bdb97cf702027305 out of 1595
cancer - 113
processing patient # 114 - 149ff4b37a684a73869d2d88e3fd4d0e out of 1595
processing patient # 115 - 14afefc82d992018c485949285d20c03 out of 1595
processing patient # 116 - 14c534e0b7c3176d9106c6215d0aa8c6 out of 1595
processing patient # 117 - 14f713c1ef037f6c531cffdff0e5fb2c out of 1595
cancer - 117
processing patient # 118 - 154a79706bcecd0402b913d4bee9eed7 out of 1595
processing patient # 119 - 155893351afed6c68893397d19555fd8 out of 1595
processing patient # 120 - 159bc8821a2dc39a1e770cb3559e098d out of 1595
Data is unlabeled
processing patient # 121 - 15aa585fb2d3018b295df8619f2d1cf7 out of 1595
cancer - 121
processing patient # 122 - 1615ace332b9d79be4e0e9fa32c71e9e out of 1595
processing patient # 123 - 1631637f08f27347e8f23d7a0e18c100 out of

processing patient # 222 - 27e9f8b51aeee67918221f2df98e9cbc out of 1595
processing patient # 223 - 28054f63c5a6ef24e06d8c34258babd6 out of 1595
processing patient # 224 - 281bb28a077ccfcd40ce4a543a5aea89 out of 1595
cancer - 224
processing patient # 225 - 28352e12fe29361dfd9613ed2e729192 out of 1595
processing patient # 226 - 28621a6988deb389673605447f46d3bb out of 1595
processing patient # 227 - 28649e08938386e3b8eaa4a84baf5982 out of 1595
processing patient # 228 - 28824d52b6425841bb263393c3211693 out of 1595
processing patient # 229 - 2885e3af725bc58dc1522d4bfb24bb2b out of 1595
processing patient # 230 - 2895ce2fe73dd62554326c3ab1c3b686 out of 1595
processing patient # 231 - 28a9b77a9113ce491433d3ea47fa8fc9 out of 1595
cancer - 231
processing patient # 232 - 28e29fe26140703e5bbe570f982bd112 out of 1595
cancer - 232
processing patient # 233 - 2922aac33a190fd3939de337fe3f0a34 out of 1595
processing patient # 234 - 2950df9632577cb4b3af2dbfe2086b11 out of 1595
processing patient # 235 

KeyboardInterrupt: 

In [ ]:
augData = np.load('augmentedDataNew-64-64-20.npy')
imageData = np.load('imageDataNew-64-64-20.npy')
for i in range(15):
    display_mid_slice(augData[i][0])
    display_mid_slice(imageData[i][0])

In [ ]:
lungPatients, labels = load_data()

size = Settings.size
no_slices = Settings.no_slices

imageData = []
augData = []

for num, patient in enumerate(lungPatients):
    if num % 100 == 0:
        print('Saved -', num)
    try:
        patient_data = data_processor(patient=patient, labels_df=labels, size=size, noslices=no_slices, visualize=False)
        print('processing patient # {} - {} out of {}'.format(num, patient, len(lungPatients)))
        if get_label(patient, labels) == 1:
            print('cancer - {}'.format(num))
            augmented_data = []
            for i in range(Settings.aug_counter):
                augmented_data.append(augment_patient(patient=patient, labels_df=labels, iteration=i, size=size, noslices=no_slices, visualize=False))
            for each in augmented_data:
                augData.append([each['img_data'], each['label'], each['patient']])
    
        imageData.append([patient_data['img_data'], patient_data['label'], patient_data['patient']])
    except KeyError as e:
        print('Data is unlabeled')
    except Exception as er:
        print(er)

        
##Results are saved as numpy file
np.save('imageDataNew-{}-{}-{}.npy'.format(size, size, no_slices), imageData)
np.save('augmentedDataNew-{}-{}-{}.npy'.format(size, size, no_slices), augData)
print(imageData.shape)
print(augData.shape)

In [2]:
import tensorflow as tf
import pandas as pd
import tflearn
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import matplotlib.pyplot as plt

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
imageData = np.load('imageDataNew-64-64-20.npy')
trainingData = imageData[0:-100]
validationData = imageData[-100:]

augData = np.load('augmentedDataNew-64-64-20.npy')
trainingData.append(augData)

x = tf.placeholder('float')
y = tf.placeholder('float')
size = 50
keep_rate = 0.8
NoSlices = 20

FileNotFoundError: [Errno 2] No such file or directory: 'imageDataNew-50-50-20.npy'

In [ ]:
def convolution3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')


def maxpooling3d(x):
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


def cnn(x):
    x = tf.reshape(x, shape=[-1, size, size, NoSlices, 1])
    convolution1 = tf.nn.relu(
        convolution3d(x, tf.Variable(tf.random_normal([3, 3, 3, 1, 32]))) + tf.Variable(tf.random_normal([32])))
    convolution1 = maxpooling3d(convolution1)
    convolution2 = tf.nn.relu(
        convolution3d(convolution1, tf.Variable(tf.random_normal([3, 3, 3, 32, 64]))) + tf.Variable(
            tf.random_normal([64])))
    convolution2 = maxpooling3d(convolution2)
    convolution3 = tf.nn.relu(
        convolution3d(convolution2, tf.Variable(tf.random_normal([3, 3, 3, 64, 128]))) + tf.Variable(
            tf.random_normal([128])))
    convolution3 = maxpooling3d(convolution3)
    convolution4 = tf.nn.relu(
        convolution3d(convolution3, tf.Variable(tf.random_normal([3, 3, 3, 128, 256]))) + tf.Variable(
            tf.random_normal([256])))
    convolution4 = maxpooling3d(convolution4)
    convolution5 = tf.nn.relu(
        convolution3d(convolution4, tf.Variable(tf.random_normal([3, 3, 3, 256, 512]))) + tf.Variable(
            tf.random_normal([512])))
    convolution5 = maxpooling3d(convolution4)
    fullyconnected = tf.reshape(convolution5, [-1, 1024])
    fullyconnected = tf.nn.relu(
        tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 1024]))) + tf.Variable(tf.random_normal([1024])))
    fullyconnected = tf.nn.dropout(fullyconnected, keep_rate)
    output = tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 2]))) + tf.Variable(tf.random_normal([2]))
    return output


def network(x):
    prediction = cnn(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    epochs = 10
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            epoch_loss = 0
            for data in trainingData:
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = session.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                except Exception as e:
                    pass

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
           # if tf.argmax(prediction, 1) == 0:
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Epoch', epoch + 1, 'completed out of', epochs, 'loss:', epoch_loss)
            # print('Correct:',correct.eval({x:[i[0] for i in validationData], y:[i[1] for i in validationData]}))
            print('Accuracy:', accuracy.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}))
        print('Final Accuracy:', accuracy.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}))
        patients = []
        actual = []
        predicted = []

        finalprediction = tf.argmax(prediction, 1)
        actualprediction = tf.argmax(y, 1)
        for i in range(len(validationData)):
            patients.append(validationData[i][2])
        for i in finalprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}):
            if(i==1):
                predicted.append("Cancer")
            else:
                predicted.append("No Cancer")
        for i in actualprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}):
            if(i==1):
                actual.append("Cancer")
            else:
                actual.append("No Cancer")
        for i in range(len(patients)):
            print("Patient: ",patients[i])
            print("Actual: ", actual[i])
            print("Predcited: ", predicted[i])

        from sklearn.metrics import confusion_matrix
        y_actual = pd.Series(
            (actualprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]})),
            name='Actual')
        y_predicted = pd.Series(
            (finalprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]})),
            name='Predicted')
        df_confusion = pd.crosstab(y_actual, y_predicted)
        print(df_confusion)

        ## Function to plot confusion matrix
        def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):\
            
            plt.matshow(df_confusion, cmap=cmap)  # imshow  
            # plt.title(title)
            plt.colorbar()
            tick_marks = np.arange(len(df_confusion.columns))
            plt.xticks(tick_marks, df_confusion.columns, rotation=45)
            plt.yticks(tick_marks, df_confusion.index)
            # plt.tight_layout()
            plt.ylabel(df_confusion.index.name)
            plt.xlabel(df_confusion.columns.name)
            plt.show()
        plot_confusion_matrix(df_confusion)
        # print(y_true,y_pred)
        # print(confusion_matrix(y_true, y_pred))
        # print(actualprediction.eval({x:[i[0] for i in validationData], y:[i[1] for i in validationData]}))
        # print(finalprediction.eval({x:[i[0] for i in validationData], y:[i[1] for i in validationData]}))
network(x)

In [ ]:
import tensorflow as tf
import pandas as pd
import tflearn
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

imageData = np.load('imageDataNew-50-50-20.npy')
trainingData = imageData[0:800]
testData = imageData[-1:]

x = tf.placeholder('float')
y = tf.placeholder('float')
size = 50
keep_rate = 0.8
NoSlices = 20


def convolution3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')


def maxpooling3d(x):
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


def cnn(x):
    x = tf.reshape(x, shape=[-1, size, size, NoSlices, 1])
    convolution1 = tf.nn.relu(
        convolution3d(x, tf.Variable(tf.random_normal([3, 3, 3, 1, 32]))) + tf.Variable(tf.random_normal([32])))
    convolution1 = maxpooling3d(convolution1)
    convolution2 = tf.nn.relu(
        convolution3d(convolution1, tf.Variable(tf.random_normal([3, 3, 3, 32, 64]))) + tf.Variable(
            tf.random_normal([64])))
    convolution2 = maxpooling3d(convolution2)
    convolution3 = tf.nn.relu(
        convolution3d(convolution2, tf.Variable(tf.random_normal([3, 3, 3, 64, 128]))) + tf.Variable(
            tf.random_normal([128])))
    convolution3 = maxpooling3d(convolution3)
    convolution4 = tf.nn.relu(
        convolution3d(convolution3, tf.Variable(tf.random_normal([3, 3, 3, 128, 256]))) + tf.Variable(
            tf.random_normal([256])))
    convolution4 = maxpooling3d(convolution4)
    convolution5 = tf.nn.relu(
        convolution3d(convolution4, tf.Variable(tf.random_normal([3, 3, 3, 256, 512]))) + tf.Variable(
            tf.random_normal([512])))
    convolution5 = maxpooling3d(convolution4)
    fullyconnected = tf.reshape(convolution5, [-1, 1024])
    fullyconnected = tf.nn.relu(
        tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 1024]))) + tf.Variable(tf.random_normal([1024])))
    fullyconnected = tf.nn.dropout(fullyconnected, keep_rate)
    output = tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 2]))) + tf.Variable(tf.random_normal([2]))
    return output


def network(x):
    prediction = cnn(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    epochs = 10
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            epoch_loss = 0
            for data in trainingData:
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = session.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                except Exception as e:
                    pass

        patients = []
        actual = []
        predicted = []

        finalprediction = tf.argmax(prediction, 1)
        actualprediction = tf.argmax(y, 1)
        for i in range(len(testData)):
            patients.append(testData[i][2])
        for i in finalprediction.eval({x: [i[0] for i in testData], y: [i[1] for i in testData]}):
            if(i==1):
                predicted.append("Cancer")
            else:
                predicted.append("No Cancer")

        for i in range(len(patients)):
            print("Patient: ",patients[i])
            print("Predcited: ", predicted[i])

     
network(x)